In [1]:
%pip install booknlp --quiet

     |████████████████████████████████| 2.4 MB 8.8 MB/s 
     |████████████████████████████████| 5.5 MB 11.2 MB/s 
     |████████████████████████████████| 182 kB 16.9 MB/s 
     |████████████████████████████████| 7.6 MB 45.3 MB/s 


In [ ]:
from booknlp.booknlp import BookNLP
import pandas as pd
from pprint import pprint
import json

In [3]:
model_params={
		"pipeline":"entity,quote,supersense,event,coref", 
		"model":"big"
	}

booknlp=BookNLP("en", model_params)

{'pipeline': 'entity,quote,supersense,event,coref', 'model': 'big'}
downloading entities_google_bert_uncased_L-6_H-768_A-12-v1.0.model
downloading coref_google_bert_uncased_L-12_H-768_A-12-v1.0.model
downloading speaker_google_bert_uncased_L-12_H-768_A-12-v1.0.1.model


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/384 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/270M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

--- startup: 43.272 seconds ---


In [ ]:
input_file="crime_and_punishment.txt"
output_directory="crime_and_punishment"
book_id="crime_and_punishment"

booknlp.process(input_file, output_directory, book_id)

--- spacy: 38.515 seconds ---
--- entities: 894.837 seconds ---
--- quotes: 0.591 seconds ---
Cannot resolve quotation
--- attribution: 2130.899 seconds ---
--- name coref: 0.782 seconds ---
--- coref: 1183.279 seconds ---
--- TOTAL (excl. startup): 4251.203 seconds ---, 256401 words


In [99]:
import sys
import csv
# wide future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

csv.field_size_limit(sys.maxsize)

9223372036854775807

In [107]:
with open('crime_and_punishment/crime_and_punishment.book') as f:
    data = json.load(f)

len(data["characters"])

1474

In [108]:
data["characters"][0].keys()

dict_keys(['agent', 'patient', 'mod', 'poss', 'id', 'g', 'count', 'mentions'])

In [101]:
tokens = pd.read_csv("crime_and_punishment/crime_and_punishment.tokens", delimiter="\t", engine="python", encoding='utf-8', error_bad_lines=False)
tokens.head()

Skipping line 130152: unexpected end of data


,paragraph_ID,sentence_ID,token_ID_within_sentence,token_ID_within_document,word,lemma,byte_onset,byte_offset,POS_tag,fine_POS_tag,dependency_relation,syntactic_head_ID,event
0,0,0,0,0,﻿The,﻿the,0,4,PRON,DT,nmod,3,O
1,0,0,1,1,Project,Project,5,12,PROPN,NNP,compound,3,O
2,0,0,2,2,Gutenberg,Gutenberg,13,22,PROPN,NNP,compound,3,O
3,0,0,3,3,eBook,eBook,23,28,PROPN,NNP,nsubj,14,O
4,0,0,4,4,of,of,29,31,ADP,IN,prep,3,O


In [16]:
entities = pd.read_csv("crime_and_punishment/crime_and_punishment.entities", delimiter="\t")
entities.head()

,COREF,start_token,end_token,prop,cat,text
0,111,10,11,PROP,PER,Fyodor Dostoevsky
1,1,22,24,PROP,LOC,the United States
2,355,26,31,NOM,LOC,most other parts of the world
3,356,30,31,NOM,LOC,the world
4,357,42,42,PRON,PER,You


In [17]:
quote = pd.read_csv("crime_and_punishment/crime_and_punishment.quotes", delimiter="\t")
quote.head()

,quote_start,quote_end,mention_start,mention_end,mention_phrase,char_id,quote
0,283,287,275,275,he,111,“ Poor Folk . ”
1,374,405,408,408,I.,121,“ taking part in conversations against the cen...
2,411,417,422,422,him,111,"“ stern and just man , ”"
3,466,580,463,463,Dostoevsky,111,"“ They snapped words over our heads , and they..."
4,704,708,663,663,He,111,"“ Dead House , ”"


In [18]:
supsense = pd.read_csv("crime_and_punishment/crime_and_punishment.supersense", delimiter="\t")
supsense.head()

,start_token,end_token,supersense_category,text
0,2,2,noun.communication,Gutenberg
1,7,7,noun.act,Punishment
2,10,11,noun.person,Fyodor Dostoevsky
3,14,14,verb.stative,is
4,17,17,noun.act,use


## Analyze characters

Code source: https://booknlp.pythonhumanities.com/04_character_analysis.html

In [70]:
import json
from collections import Counter

def proc(filename):
    with open(filename) as file:
        data=json.load(file)
    return data

def get_counter_from_dependency_list(dep_list):
    counter=Counter()
    for token in dep_list:
        term=token["w"]
        tokenGlobalIndex=token["i"]
        counter[term]+=1
    return counter


def create_character_data(data, printTop):
    character_data = {}
    for character in data["characters"]:

        agentList=character["agent"]
        patientList=character["patient"]
        possList=character["poss"]
        modList=character["mod"]

        character_id=character["id"]
        count=character["count"]

        referential_gender_distribution=referential_gender_prediction="unknown"

        if character["g"] is not None and character["g"] != "unknown":
            referential_gender_distribution=character["g"]["inference"]
            referential_gender=character["g"]["argmax"]

        mentions=character["mentions"]
        proper_mentions=mentions["proper"]
        max_proper_mention=""
        
        #Let's create some empty lists that we can append to.
        poss_items = []
        agent_items = []
        patient_items = []
        mod_items = []
    
        # just print out information about named characters
        if len(mentions["proper"]) > 0:
            max_proper_mention=mentions["proper"][0]["n"]
            for k, v in get_counter_from_dependency_list(possList).most_common(printTop):
                poss_items.append((v,k))
                
            for k, v in get_counter_from_dependency_list(agentList).most_common(printTop):
                agent_items.append((v,k))     

            for k, v in get_counter_from_dependency_list(patientList).most_common(printTop):
                patient_items.append((v,k))     

            for k, v in get_counter_from_dependency_list(modList).most_common(printTop):
                mod_items.append((v,k))  

            
            
            
            # print(character_id, count, max_proper_mention, referential_gender)
            character_data[character_id] = {"id": character_id,
                                  "count": count,
                                  "max_proper_mention": max_proper_mention,
                                  "referential_gender": referential_gender,
                                  "possList": poss_items,
                                  "agentList": agent_items,
                                  "patientList": patient_items,
                                  "modList": mod_items
                                 }
                                
    return character_data


def find_verb_usage(character_data, analysis=["agent", "patient"]):
    new_analysis = []
    for item in analysis:
        if item == "agent":
            new_analysis.append("agentList")
        elif item == "patient":
            new_analysis.append("patientList")
    main_agents = {}
    main_patients = {}
    for character in character_data:
        temp_data = character_data[character]
        for item in new_analysis:
            for verb in temp_data[item]:
                verb = verb[1].lower()
                if item == "agentList":
                    if verb not in main_agents:
                        main_agents[verb] = [(character, temp_data["max_proper_mention"])]
                    else:
                        main_agents[verb].append((character, temp_data["max_proper_mention"]))
                elif item == "patientList":
                    if verb not in main_patients:
                        main_patients[verb] = [(character, temp_data["max_proper_mention"])]
                    else:
                        main_patients[verb].append((character, temp_data["max_proper_mention"]))
    verb_usage = {"agent": main_agents,
                 "patient": main_patients}
    return verb_usage


In [72]:
data=proc("crime_and_punishment/crime_and_punishment.book")
character_data = create_character_data(data, 20)

In [73]:
pprint(character_data)

{111: {'agentList': [(2, 'contrived'),
                     (2, 'started'),
                     (2, 'became'),
                     (1, 'begun'),
                     (1, 'says'),
                     (1, 'concluded'),
                     (1, 'had'),
                     (1, 'thought'),
                     (1, 'bid'),
                     (1, 'recurs'),
                     (1, 'describes'),
                     (1, 'insists'),
                     (1, 'began'),
                     (1, 'shown'),
                     (1, 'suffered'),
                     (1, 'lost'),
                     (1, 'took'),
                     (1, 'made'),
                     (1, 'received'),
                     (1, 'died')],
       'count': 72,
       'id': 111,
       'max_proper_mention': 'Dostoevsky',
       'modList': [(1, 'son'), (1, 'writer')],
       'patientList': [(1, 'found'),
                       (1, 'arrested'),
                       (1, 'accused'),
                       (1, 'calls'),
 

In [95]:
for cid, character in list(character_data.items())[:10]:
    print(f"{cid} - {character['max_proper_mention']}")

131 - Raskolnikov
161 - Raskolnikov
249 - Sonia
217 - Razumihin
213 - Porfiry
154 - Dounia
156 - Svidrigaïlov
153 - Rodya
164 - Avdotya Romanovna
138 - Katerina Ivanovna


In [75]:
verb_data = find_verb_usage(character_data)

In [83]:
pprint(list(verb_data["agent"].keys())[:40], compact=True)

['said', 'know', 'felt', 'went', 'had', 'thought', 'looked', 'come', 'want',
 'have', 'believe', 'walked', 'began', 'heard', 'think', 'tell', 'made',
 'stood', 'turned', 'asked', 'understand', 'saw', 'cried', 'see', 'answered',
 'go', 'seen', 'gave', 'sat', 'knew', 'came', 'whispered', 'got', 'told',
 'talking', 'took', 'give', 'let', 'say', 'has']


In [84]:
pprint(list(verb_data["patient"].keys())[:40], compact=True)

['tell', 'see', 'told', 'asked', 'cried', 'struck', 'left', 'arrest', 'take',
 'thought', 'excuse', 'betray', 'addressing', 'saw', 'put', 'taking', 'drew',
 'know', 'believe', 'assure', 'taken', 'worry', 'killed', 'forgive', 'blamed',
 'exclaimed', 'added', 'crossing', 'observed', 'crushed', 'reassure',
 'watching', 'whispered', 'love', 'beg', 'followed', 'accompanied', 'find',
 'ask', 'worried']


In [81]:
verb_data["agent"]["died"]

[(157, 'Marfa Petrovna'), (111, 'Dostoevsky')]

In [79]:
verb_data["agent"]["lies"]

[(215, 'Dushkin')]

In [59]:
verb_data["agent"]["kill"]

[(216, 'Nikolay'), (176, 'Mikolka'), (189, 'Karl')]

In [60]:
verb_data["agent"]["thrashed"]

[(176, 'Mikolka')]

In [65]:
verb_data["agent"]["love"]

[(239, 'Polenka'), (166, 'Mother')]

In [90]:
verb_data["agent"]["loves"]

[(154, 'Dounia'), (199, 'Pashenka')]

In [62]:
verb_data["patient"]["tempted"]

[(154, 'Dounia'), (181, 'Koch')]

In [87]:
verb_data["patient"]["blow"]

[(186, 'Tchebarov')]

In [88]:
verb_data["patient"]["betray"]

[(131, 'Raskolnikov')]

In [69]:
verb_data["patient"]["stabbed"]

[(249, 'Sonia')]

In [89]:
verb_data["patient"]["struck"]

[(131, 'Raskolnikov'),
 (161, 'Raskolnikov'),
 (164, 'Avdotya Romanovna'),
 (159, 'Pyotr Petrovitch'),
 (207, 'Zametov'),
 (137, 'Lebeziatnikov'),
 (267, 'Raskolnikov'),
 (278, 'Napoleon'),
 (339, 'Achilles')]

## event analysis

code source: https://booknlp.pythonhumanities.com/05_events.html#

In [25]:
events = tokens[~tokens['event'].isnull()]
events

,paragraph_ID,sentence_ID,token_ID_within_sentence,token_ID_within_document,word,lemma,byte_onset,byte_offset,POS_tag,fine_POS_tag,dependency_relation,syntactic_head_ID,event
0,0,0,0,0,﻿The,﻿the,0,4,PRON,DT,nmod,3,O
1,0,0,1,1,Project,Project,5,12,PROPN,NNP,compound,3,O
2,0,0,2,2,Gutenberg,Gutenberg,13,22,PROPN,NNP,compound,3,O
3,0,0,3,3,eBook,eBook,23,28,PROPN,NNP,nsubj,14,O
4,0,0,4,4,of,of,29,31,ADP,IN,prep,3,O
...,...,...,...,...,...,...,...,...,...,...,...,...,...
130145,3976,13448,3,252853,her,her,1135119,1135122,PRON,PRP$,poss,252854,O
130146,3976,13448,4,252854,aspirations,aspiration,1135123,1135134,NOUN,NNS,npadvmod,252864,O
130147,3976,13448,5,252855,at,at,1135135,1135137,ADP,IN,prep,252854,O
130148,3976,13448,6,252856,least,least,1135138,1135143,ADJ,JJS,pobj,252855,O


In [27]:
real_events = events.loc[events["event"] == "EVENT"]
real_events

,paragraph_ID,sentence_ID,token_ID_within_sentence,token_ID_within_document,word,lemma,byte_onset,byte_offset,POS_tag,fine_POS_tag,dependency_relation,syntactic_head_ID,event
266,16,12,12,266,examination,examination,1423,1434,NOUN,NN,pobj,263,EVENT
287,17,14,3,291,published,publish,1548,1557,VERB,VBN,ROOT,291,EVENT
336,17,17,4,340,arrested,arrest,1813,1821,VERB,VBN,ROOT,340,EVENT
358,18,18,20,362,met,meet,1940,1943,VERB,VBD,relcl,360,EVENT
368,18,19,2,372,accused,accuse,1990,1997,VERB,VBN,ROOT,372,EVENT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
129186,3946,13251,36,249121,wrote,write,1117304,1117309,VERB,VBD,ROOT,249121,EVENT
129310,3947,13254,5,249245,wrote,write,1117918,1117923,VERB,VBD,ROOT,249245,EVENT
129909,3952,13288,11,249847,thought,think,1120696,1120703,VERB,VBD,ROOT,249847,EVENT
129912,3952,13288,14,249850,criticised,criticise,1120713,1120723,VERB,VBD,conj,249847,EVENT


In [30]:
def get_event_sentences(df):
    real_events = df.loc[df["event"] == "EVENT"]
    sentences = real_events.sentence_ID.tolist()
    event_words = real_events.word.tolist()
    event_lemmas = real_events.lemma.tolist()
    final_sentences = []
    for i, sentence in enumerate(sentences):
        result = df[df["sentence_ID"] == int(sentence)]
        words = result.word.tolist()
        resentence = " ".join(words)
        final_sentences.append(
            {
                "event_word": event_words[i],
                "event_lemma": event_lemmas[i],
                "sentence": resentence,
            }
        )
    return pd.DataFrame(final_sentences)

In [33]:
events = get_event_sentences(tokens)
events

,event_word,event_lemma,sentence
0,examination,examination,Though always sickly and delicate Dostoevsky c...
1,published,publish,This story was published by the poet Nekrassov...
2,arrested,arrest,In 1849 he was arrested .
3,met,meet,Though neither by temperament nor conviction a...
4,accused,accuse,He was accused of “
...,...,...,...
5987,wrote,write,On working days she would go to see him at wor...
5988,wrote,write,In the last letter Sonia wrote that he had bee...
5989,thought,think,"But now in prison , _ in freedom _ , he though..."
5990,criticised,criticise,"But now in prison , _ in freedom _ , he though..."


In [66]:
pprint(list(events["event_word"].unique()), compact=True)

['examination', 'published', 'arrested', 'met', 'accused', 'condemned', 'taken',
 'says', 'commuted', 'mad', 'untied', 'arrest', 'attacks', 'fits', 'allowed',
 'started', 'speech', 'unveiling', 'demonstrations', 'died', 'followed', 'came',
 'lodged', 'walked', 'feeling', 'given', 'coming', 'aware', 'fears', 'thought',
 'smile', 'heat', 'completed', 'expression', 'confessed', 'dragged', 'shouted',
 'drove', 'bawling', 'pointing', 'stopped', 'clutched', 'overtaken', 'muttered',
 'counted', 'tantalising', 'look', 'monologues', 'jeered', 'regard', 'dream',
 'excitement', 'tremor', 'went', 'going', 'glad', 'slipped', 'asking',
 'reached', 'progress', 'barred', 'moving', 'rang', 'tinkle', 'overstrained',
 'opened', 'eyed', 'glittering', 'seeing', 'bolder', 'stepped', 'facing',
 'looking', 'flapping', 'coughed', 'groaned', 'gleam', 'haste', 'mutter', 'bow',
 'remembering', 'said', 'inquiring', 'continued', 'disconcerted', 'surprised',
 'mistrust', 'paused', 'letting', 'pass', 'lighted', 'sett

In [65]:
pprint(events[events["event_word"] == "confess"]["sentence"].values)

array(['He knows , ” Forgive my troubling you about such trifles , ” the things are only worth five roubles , but I prize them particularly for the sake of those from whom they came to me , and I must confess that I was alarmed when I heard ... ” That ’s why you were so much struck when I mentioned to Zossimov that Porfiry was inquiring for everyone who had pledges ! ”',
       'How did he strike you ? ” I must confess I do n’t quite understand him .',
       'And although I can not , I confess , in principle sympathise with private charity , for it not only fails to eradicate the evil but even promotes it , yet I must admit that I saw your action with pleasure -- yes , yes , I like it . ” That ’s all nonsense , ”',
       'We ’ll speak of children later , but now as to the question of honour , I confess that ’s my weak point .',
       'To the question what led him to confess , he answered that it was his heartfelt repentance .'],
      dtype=object)


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a605a3e6-1564-47b2-94e7-842290ba7692' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>